In [1]:
using DiffEqFlux, OrdinaryDiffEq, Flux, Optim
include("/Users/piotrsokol/Documents/RNNODE.jl/src/rnn_ode.jl")
using Zygote
using Flux: logitcrossentropy
using Flux.Data: DataLoader
using MLDatasets, NNlib, MLDataUtils
#ENV["PYTHON"] = "/Users/piotrsokol/anaconda3/envs/bortho/bin/python"
#using Pkg; Pkg.build("PyCall")
using PyCall
using CUDA
# using Parameters: @with_kw, @unpack
import Statistics: mean, std
using BSON,NPZ
using UUIDs
using EllipsisNotation
using ProgressMeter
using ArgParse
FT = Float32

Float32

In [2]:
    py"""
    import sys
    sys.path.insert(0, "/Users/piotrsokol/Documents/block-orthogonal/src/")
    """

In [3]:
function onehot(labels_raw; ntoken::Int=9)
    return  convertlabel(LabelEnc.OneOfK, labels_raw, LabelEnc.NativeLabels(collect(0:ntoken)))
end

function preprocess_img(imgs, p)
    imgs = reshape( imgs, prod(size(imgs)[1:2]), size(imgs)[3] ) |> x-> FT.(x)
    imgs = map(x-> permute!(x,p), eachcol(imgs)) |> x-> hcat(x...)
    imgs .-= 0.13066047
    imgs./= 0.30810785
    return imgs
end
function get_data(batchsize, device, set::Symbol; train_split=58999/60000)
    py_seeds= pyimport("data_utils").__fixed_seeds__
    py_random=pyimport("numpy.random");
    seeded_rng = py_random.RandomState(py_seeds["image"])
    p = seeded_rng.permutation(1:784)

    X, labels_raw = MNIST.traindata()
    Y = onehot(labels_raw)
    X = preprocess_img(X, p)

    (x_train, y_train), (x_valid, y_valid) = stratifiedobs((X, Y),p = train_split)
    
    if set!= :test
        return (
                DataLoader(device.(collect.((x_train, y_train))); batchsize = batchsize, shuffle = true),
                DataLoader(device.(collect.((x_valid, y_valid))); batchsize = batchsize, shuffle = false) )
    elseif set == :test
        X, labels_raw = MNIST.testdata()
        Y = onehot(labels_raw)
        X = preprocess_img(X, p)
        return (
                DataLoader(device.(collect.((x_train, y_train))); batchsize = batchsize, shuffle = true),
                DataLoader(device.(collect.((x_valid, y_valid))); batchsize = batchsize, shuffle = false),
                DataLoader(device.(collect.((X, Y))); batchsize = batchsize, shuffle = false)
            )
    end
end

get_data (generic function with 1 method)

In [4]:
tr_loader, valid_loader = get_data(100, cpu, :valid);

In [5]:
X,Y = first(tr_loader);

In [6]:
function get_network(alpha, architecture, initializer, isize, hsize,osize, tsteps, interpolation)
    if architecture == "RNN_TANH"
        ∂rnncell = ∂RNNCell
    elseif architecture == "GRU"
        ∂rnncell = ∂GRUCell
    else
        ∂rnncell = ∂LSTMCell
    end
    """
    Ternary op -> reads as if initializer == "limitcycle" or architecture == "LSTM" use two argument function dispatch, else additionally pass initializer variable
    """
    ∂rnn = initializer == "limitcycle" || architecture == "LSTM" ? ∂rnncell(isize, hsize) : ∂rnncell(isize,hsize,Flux.glorot_uniform)

    node = RNNODE(∂rnn, (0.f0, tsteps[end]), preprocess=x-> FT.(permutedims(x)), save_end=true, save_start=false, saveat=collect(0.f0:tsteps[end]) )

    function interpolate(x)
        X = Zygote.ignore() do
            permutedims(x) |> interpolation
        end
    end
    return Chain( interpolate, node, x->x(tsteps[end]), Dense(hsize, osize) )
end

get_network (generic function with 1 method)

In [7]:
nn = get_network(1.5f0, "RNN_TANH", "limitcycle", 1, 256, 10, 784, CubicSplineFixedGrid);

In [8]:
yhat = nn(X)

10×100 Array{Float32,2}:
  1.84861     1.34054      0.829864   …   0.724764   1.63817    1.95265
 -0.189288    0.607541     0.555655      -0.104347   0.662509   0.656207
 -0.0441701  -0.0232721   -1.04787       -0.752683   0.166954  -0.143824
 -0.350011   -0.432372     0.0776971     -0.232457  -0.202694  -0.646513
 -0.14678    -0.00269686   1.38608        0.476263   0.45857   -0.446222
  1.43147     0.246254    -0.321945   …  -0.387105   0.414585   0.850229
  0.891496    1.03181      0.954996       0.797941   0.265754   0.401042
 -1.28737    -0.536155    -0.774717      -0.320387  -0.975638  -0.710939
  1.39438     0.972185     1.80663        1.26954    1.31069    0.757936
 -0.741829   -0.939822    -0.758118      -0.888271  -1.35411   -0.479881

In [22]:
interpolation= "CubicSplineFixedGrid"
itp = interpolation

"CubicSplineFixedGrid"

In [27]:
:(Symbol($itp)) |>eval

:CubicSplineFixedGrid

In [28]:
Symbol(itp)

:CubicSplineFixedGrid

CubicSplineFixedGrid